In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
def get_dataframe(root_fldr, weight=1, manual=1):
    fldrs = os.listdir(root_fldr)
    full_df = pd.DataFrame(columns=['name', 'weight', 'grove', 'manual', 'folder'])
    for folder in fldrs:
        fldr_path = os.path.join(root_fldr, folder)
        if not os.path.isdir(fldr_path):
            continue
        inside = os.listdir(fldr_path)
        msk_dir = 'masks'
        if os.path.exists(os.path.join(fldr_path, msk_dir)):
            path_msk = os.path.join(fldr_path, msk_dir)
            filenames = os.listdir(path_msk)
            weights = np.full((len(filenames)), weight)
            df = pd.DataFrame(columns=['name', 'weight', 'grove', 'manual', 'folder'])
            # df['name'] = [os.path.join(path_msk, n) for n in filenames]
            df['name'] = filenames
            df['weight'] = weights
            df['manual'] = manual
            df['folder'] = folder
            full_df = pd.concat([full_df, df])

#     full_df['grove'] = [i.split('/')[-1].split('_')[0] if i.split('/')[-1].split('_')[1] == 'nrg' else i.split('/')[-1].split('_')[0] + '_' + i.split('/')[-1].split('_')[1] for i in full_df['name']]
    full_df['grove'] = full_df['folder']
    return full_df

In [3]:
root_ = "/mnt/storage_4tb/ymi/geo_data/angle_net_data/dist_maps_train"
first_df = get_dataframe(root_, weight=1, manual=1)

In [4]:
first_df['grove'].nunique()

27

In [5]:
len(first_df)

1570

In [6]:
# Agrowing
train_man_agr, test_agr = train_test_split(first_df, test_size=0.1, random_state=0, stratify=first_df[['grove']])
#train_only_man_agr, val_only_man_agr = train_test_split(train_man_agr, test_size=0.1, random_state=0, stratify=first_df[['grove']])

In [7]:
train_man_agr.to_csv("/mnt/storage_4tb/ymi/geo_data/angle_net_data/dist_maps_train/train_1570_dist.csv", index=False)
test_agr.to_csv("/mnt/storage_4tb/ymi/geo_data/angle_net_data/dist_maps_train/val_1570_dist.csv", index=False)

In [33]:
np.round(10.1)

10.0

In [34]:
np.arange(0,10.1,1)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [36]:
np.linspace(0, 10.5, np.round(10.5) + 1, endpoint=10.5)

array([ 0.  ,  1.05,  2.1 ,  3.15,  4.2 ,  5.25,  6.3 ,  7.35,  8.4 ,
        9.45, 10.5 ])

In [16]:
def create_raster_list(root_fldr):
    """Function that creates raster list for the processing

    :param root_fldr: Path to the project folder with rasters
    :type root_fldr: string
    :return: Dictionary with raster types as keys and paths to them as values
    :rtype: dict
    """

    raster_list = {}
    channels = None
    files = [f for f in os.listdir(root_fldr) if os.path.isfile(os.path.join(root_fldr, f))]
    for file_name in files:
        name = file_name.replace(".tif", "")
        print("create list : %s" % name)
        raster_path = os.path.join(root_fldr, file_name)

        if name == "project_transparent_mosaic_group1":
            raster_list["rgb"] = raster_path
        else:
            raster_name = name.replace(" ", "_")
            raster_name = raster_name.split("_")[-1]
            raster_list[raster_name] = raster_path
    return raster_list

In [23]:
import json
root_ = "/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122"
rl = create_raster_list(root_)
print(rl)

create list : project_transparent_mosaic_red
create list : project_transparent_mosaic_nir
create list : project_transparent_mosaic_green
create list : rl__1-122
create list : project_transparent_mosaic_blue
{'red': '/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122/project_transparent_mosaic_red.tif', 'blue': '/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122/project_transparent_mosaic_blue.tif', '1-122': '/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122/rl__1-122', 'nir': '/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122/project_transparent_mosaic_nir.tif', 'green': '/mnt/storage_4tb/ymi/geo_data/angle_net_data/cs_rio_pardo/1-122/project_transparent_mosaic_green.tif'}


In [22]:
json_path = os.path.join(root_, "_".join(["rl_", os.path.basename(root_)]))

with open(json_path, "w") as dst:
    json.dump(rl, dst)